<a href="https://colab.research.google.com/github/raznuss/global_urban_flood_database/blob/main/GFD_GHSL_intersact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run all GFD events in sequential batches and export clusters with urban flooding only
# Each batch creates one CSV on Google Drive, with fixed column order and no geometry

import ee, geemap, math, time, gc
from datetime import datetime, timezone

# EE init
PROJECT_ID = "iconic-exchange-477008-h2"
try:
    ee.Initialize(project=PROJECT_ID)
except Exception:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
print("EE initialized for project:", PROJECT_ID)

# Parameters
URBAN_THRESHOLD   = 0.20     # built fraction threshold to define urban
AREA_SCALE_M      = 500      # scale for area/centroid sums (coarser = cheaper)
LABEL_SCALE_M     = 2000     # scale for connected components labeling
TILE_SCALE        = 12
BATCH_SIZE        = 50       # events per batch (adjust as you wish)
SLEEP_SEC         = 10       # polling interval
START_BATCH_INDEX = 0        # start here
END_BATCH_INDEX   = None     # None = run all

# Optional date filter (None for all)
DATE_FILTER = None
# DATE_FILTER = ('2000-01-01', '2002-12-31')

# Datasets
GFD     = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1').sort('system:time_start')
GHSL_IC = ee.ImageCollection('JRC/GHSL/P2023A/GHS_BUILT_S')
if DATE_FILTER:
    GFD = GFD.filterDate(*DATE_FILTER)

# Final column order for CSV
FINAL_FIELDS = [
    'event_id',
    'time_start_utc',
    'time_end_utc',
    'duration_days',
    'countries',
    'dfo_main_cause',
    'dfo_severity',
    'dfo_dead',
    'dfo_displaced',
    'dfo_validation_type',
    'composite_type',
    'ghsl_year_used',
    'urban_threshold_frac',
    'urban_area_m2',
    'urban_flood_area_m2',
    'flood_share_urban_pct',
    'cluster_centroid_xy_round'
]

# Helpers
def nearest_ghsl_year(evt_year):
    yrs = ee.List.sequence(1975, 2030, 5)
    fc  = ee.FeatureCollection(yrs.map(lambda y: ee.Feature(None, {
        'year': ee.Number(y).toInt(),
        'diff': ee.Number(y).subtract(evt_year).abs()
    })))
    return ee.Number(ee.Feature(fc.sort('diff').first()).get('year')).toInt()

def ghsl_image_for_year(y):
    y = ee.Number(y).toInt()
    img = GHSL_IC.filter(ee.Filter.eq('system:index', ee.String(y.format()))).first()
    fallback = GHSL_IC.filter(ee.Filter.eq('system:index', '2020')).first()
    return ee.Image(ee.Algorithms.If(img, img, fallback))

def event_times(img):
    # returns (start_date, end_date_or_None, duration_days_or_None)
    start_ms = img.get('system:time_start')
    end_ms   = img.get('system:time_end')
    start_dt = ee.Date(start_ms)
    end_dt   = ee.Algorithms.If(end_ms, ee.Date(end_ms), None)
    dur_days = ee.Algorithms.If(end_ms,
                                ee.Date(end_ms).difference(start_dt, 'day'),
                                None)
    return (start_dt, end_dt, dur_days)

# Core per-event → FeatureCollection of clusters
def event_to_clusters(img):
    # Clip to event geometry
    geom    = img.geometry()
    flooded = img.select('flooded').eq(1).unmask(0).clip(geom)
    perm    = img.select('jrc_perm_water').eq(1).unmask(0).clip(geom)
    fnp     = flooded.And(perm.Not()).rename('fnp').unmask(0).clip(geom)

    # GHSL year and built fraction
    evt_year  = ee.Number.parse(ee.Date(img.get('system:time_start')).format('YYYY'))
    ghsl_year = nearest_ghsl_year(evt_year)
    ghsl      = ghsl_image_for_year(ghsl_year)

    built_frac = ghsl.select('built_surface').divide(10000.0).clip(geom)
    urban_mask = built_frac.gte(URBAN_THRESHOLD).rename('urban').unmask(0).clip(geom)

    # Urban flooding mask (0/1)
    urban_flood = fnp.And(urban_mask).rename('uf').unmask(0).clip(geom)

    # Label urban components on urban only, coarse scale to save compute
    proj_base = ghsl.select('built_surface').projection()
    labels = (urban_mask
              .reproject(proj_base.atScale(LABEL_SCALE_M))
              .selfMask()
              .connectedComponents(ee.Kernel.plus(1), maxSize=1024)
              .select('labels').toInt64().rename('label'))

    # Area rasters
    px_area = ee.Image.pixelArea()
    urban_area_img       = urban_mask.multiply(px_area).rename('urban_area_m2')
    urban_flood_area_img = urban_flood.multiply(px_area).rename('urban_flood_area_m2')

    # Centroid (cheap): weighted mean of lon/lat using urban_flood as weights
    lonlat = ee.Image.pixelLonLat().clip(geom)
    w      = urban_flood.rename('w')  # weights
    lon_w  = lonlat.select('longitude').multiply(w).rename('lon_w')
    lat_w  = lonlat.select('latitude').multiply(w).rename('lat_w')

    # Prepare grouped reducer:
    #   bands = [urban_area_m2, urban_flood_area_m2, lon_w, lat_w, w, label]
    group_img = ee.Image.cat([
        urban_area_img,
        urban_flood_area_img,
        lon_w,
        lat_w,
        w,
        labels
    ]).updateMask(urban_mask)

    reducers  = ee.Reducer.sum().repeat(5).group(groupField=5, groupName='label')

    sums = group_img.reduceRegion(
        reducer=reducers,
        geometry=geom,
        scale=AREA_SCALE_M,
        maxPixels=2e13,
        tileScale=TILE_SCALE
    )

    groups = ee.List(ee.Dictionary(sums).get('groups', ee.List([])))
    empty_fc = ee.FeatureCollection([])

    # safely fetch label key even if named 'group'
    def _safe_label(rec):
        rec = ee.Dictionary(rec)
        return ee.Algorithms.If(
            rec.contains('label'),
            rec.get('label'),
            ee.Algorithms.If(
                rec.contains('group'),
                rec.get('group'),
                ee.Algorithms.If(
                    rec.contains('group_2'),
                    rec.get('group_2'),
                    None
                )
            )
        )

    def rec_to_feat(rec):
        rec = ee.Dictionary(rec)
        lab_any = _safe_label(rec)

        def _mk():
            sum_list = ee.List(rec.get('sum', ee.List([0,0,0,0,0])))
            ua   = ee.Number(sum_list.get(0))   # urban_area_m2
            ufa  = ee.Number(sum_list.get(1))   # urban_flood_area_m2
            lonw = ee.Number(sum_list.get(2))
            latw = ee.Number(sum_list.get(3))
            ww   = ee.Number(sum_list.get(4))

            pct = ee.Algorithms.If(ua.gt(0), ufa.divide(ua).multiply(100.0), None)

            # centroid = (sum lon*w)/sum w , (sum lat*w)/sum w
            cen_lon = ee.Algorithms.If(ww.gt(0), lonw.divide(ww), None)
            cen_lat = ee.Algorithms.If(ww.gt(0), latw.divide(ww), None)

            # rounded single field "lon,lat" with 4 decimals; None if no weight
            cen_str = ee.Algorithms.If(
                ww.gt(0),
                ee.String(ee.Number(cen_lon).format('%.4f')).cat(',').cat(ee.String(ee.Number(cen_lat).format('%.4f'))),
                None
            )

            start_dt, end_dt, dur_days = event_times(img)
            return ee.Feature(None, {
                'event_id'                 : img.get('id'),
                'time_start_utc'           : ee.Date(start_dt).format('YYYY-MM-dd HH:mm'),
                'time_end_utc'             : ee.Algorithms.If(end_dt, ee.Date(end_dt).format('YYYY-MM-dd HH:mm'), 'N/A'),
                'duration_days'            : dur_days,
                'countries'                : img.get('countries'),
                'dfo_main_cause'           : img.get('dfo_main_cause'),
                'dfo_severity'             : img.get('dfo_severity'),
                'dfo_dead'                 : img.get('dfo_dead'),
                'dfo_displaced'            : img.get('dfo_displaced'),
                'dfo_validation_type'      : img.get('dfo_validation_type'),
                'composite_type'           : img.get('composite_type'),
                'ghsl_year_used'           : ghsl_year,
                'urban_threshold_frac'     : URBAN_THRESHOLD,
                'urban_area_m2'            : ua,
                'urban_flood_area_m2'      : ufa,
                'flood_share_urban_pct'    : pct,
                'cluster_centroid_xy_round': cen_str
            })
        return ee.Algorithms.If(ee.Algorithms.IsEqual(lab_any, None), ee.Feature(None, {}), _mk())

    fc = ee.FeatureCollection(ee.Algorithms.If(
        groups.size().gt(0),
        ee.FeatureCollection(groups.map(rec_to_feat)),
        empty_fc
    ))

    # Keep only non-empty + clusters that actually flooded in urban areas
    fc = fc.filter(ee.Filter.notNull(['urban_flood_area_m2'])) \
           .filter(ee.Filter.gt('urban_flood_area_m2', 0))

    # enforce column order now, drop any accidental extras (and geometry)
    fc = fc.select(FINAL_FIELDS)

    return fc

def export_batch(batch_index, batch_size, ic):
    """Build subset, compute clusters, filter to flooded-only, export CSV, poll to completion."""
    n_events = ic.size().getInfo()
    start_idx = batch_index * batch_size
    end_idx   = min(start_idx + batch_size, n_events)
    if start_idx >= n_events:
        raise ValueError(f"Batch {batch_index} out of range")

    sub_list = ic.toList(end_idx - start_idx, start_idx)
    sub_ic   = ee.ImageCollection(sub_list)

    sub_fc = ee.FeatureCollection(sub_ic.map(event_to_clusters)).flatten()

    size_est = sub_fc.size().getInfo()
    if size_est == 0:
        print(f"[Batch {batch_index}] Empty result, skipping export.")
        del sub_list, sub_ic, sub_fc
        gc.collect()
        return

    # simpler filename numbering
    desc = f"urban_flood_clusters_{batch_index + 1}"

    task = ee.batch.Export.table.toDrive(
        collection=sub_fc,
        description=desc,
        fileFormat='CSV',
        fileNamePrefix=desc,
        selectors=FINAL_FIELDS  # only these fields, no geometry
    )
    task.start()
    print(f"[Batch {batch_index}] Export started:", desc, f"(rows ~ {size_est})")

    # Poll until done
    while True:
        status = task.status()
        state  = status.get('state', 'UNKNOWN')
        if state in ('COMPLETED', 'FAILED', 'CANCELLED'):
            print(f"[Batch {batch_index}] Task finished with state:", state)
            if state != 'COMPLETED':
                print(f"[Batch {batch_index}] Status detail:", status)
            break
        time.sleep(SLEEP_SEC)

    del sub_list, sub_ic, sub_fc, task
    gc.collect()


# Run sequential batches
total_events  = GFD.size().getInfo()
total_batches = math.ceil(total_events / BATCH_SIZE)
if END_BATCH_INDEX is None:
    END_BATCH_INDEX = total_batches - 1

print("Total events:", total_events)
print("Batch size:", BATCH_SIZE)
print(f"Running batches {START_BATCH_INDEX} .. {END_BATCH_INDEX} (of {total_batches - 1})")

for b in range(START_BATCH_INDEX, END_BATCH_INDEX + 1):
    try:
        export_batch(b, BATCH_SIZE, GFD)
    except Exception as e:
        print(f"[Batch {b}] Error:", e)
        time.sleep(5)
        continue

print("All requested batches processed.")
print("CSV files will appear in your Google Drive under My Drive.")


EE initialized for project: iconic-exchange-477008-h2
Total events: 913
Batch size: 50
Running batches 0 .. 18 (of 18)
[Batch 0] Export started: urban_flood_clusters_1 (rows ~ 110)
[Batch 0] Task finished with state: COMPLETED
[Batch 1] Export started: urban_flood_clusters_2 (rows ~ 297)
[Batch 1] Task finished with state: COMPLETED
[Batch 2] Export started: urban_flood_clusters_3 (rows ~ 255)
[Batch 2] Task finished with state: COMPLETED
[Batch 3] Export started: urban_flood_clusters_4 (rows ~ 262)
[Batch 3] Task finished with state: COMPLETED
[Batch 4] Export started: urban_flood_clusters_5 (rows ~ 182)
[Batch 4] Task finished with state: COMPLETED
[Batch 5] Export started: urban_flood_clusters_6 (rows ~ 239)
[Batch 5] Task finished with state: COMPLETED
[Batch 6] Export started: urban_flood_clusters_7 (rows ~ 223)
[Batch 6] Task finished with state: COMPLETED
[Batch 7] Export started: urban_flood_clusters_8 (rows ~ 74)
[Batch 7] Task finished with state: COMPLETED
[Batch 8] Export s

In [ ]:
# ===== Show exactly one CSV row: urban cluster and its flooded part =====
# Choose one: by row index, or by (event_id, centroid_xy)
ROW_INDEX = 10                     # 0-based index in the CSV, set None to use ID+centroid
EVENT_ID_FILTER = None             # e.g., 1470
CENTROID_XY_FILTER = None          # e.g., "106.4694,29.5252"
FALLBACK_URBAN_THRESHOLD = 0.20    # used if CSV row has NaN in urban_threshold_frac

import ee, geemap, pandas as pd, math
from google.colab import files

# --- EE init (your project id) ---
PROJECT_ID = "iconic-exchange-477008-h2"
try:
    ee.Initialize(project=PROJECT_ID)
except Exception:
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
print("EE ready")

# --- Upload CSV from your computer ---
print("Please upload your CSV (urban_flood_clusters_*.csv)…")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")
CSV_PATH = list(uploaded.keys())[0]
print("Using:", CSV_PATH)

df = pd.read_csv(CSV_PATH)

# --- Pick exactly one row ---
row = None
if ROW_INDEX is not None:
    row = df.iloc[ROW_INDEX]
else:
    if EVENT_ID_FILTER is None or CENTROID_XY_FILTER is None:
        raise ValueError("Provide either ROW_INDEX or both EVENT_ID_FILTER and CENTROID_XY_FILTER")
    sub = df[(df["event_id"] == EVENT_ID_FILTER) &
             (df["cluster_centroid_xy_round"] == CENTROID_XY_FILTER)]
    if sub.empty:
        raise ValueError("No matching row")
    row = sub.iloc[0]

# --- Extract values from the chosen row ---
event_id = int(row["event_id"])
centroid_xy = str(row["cluster_centroid_xy_round"])
urban_thr = float(row["urban_threshold_frac"]) if "urban_threshold_frac" in row and not pd.isna(row["urban_threshold_frac"]) else FALLBACK_URBAN_THRESHOLD

# --- Helpers reused ---
GHSL_IC = ee.ImageCollection('JRC/GHSL/P2023A/GHS_BUILT_S')
GFD = ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')

def nearest_ghsl_year(evt_year):
    yrs = ee.List.sequence(1975, 2030, 5)
    fc  = ee.FeatureCollection(yrs.map(lambda y: ee.Feature(None, {
        'year': ee.Number(y).toInt(),
        'diff': ee.Number(y).subtract(evt_year).abs()
    })))
    return ee.Number(ee.Feature(fc.sort('diff').first()).get('year')).toInt()

def ghsl_image_for_year(y):
    y = ee.Number(y).toInt()
    img = GHSL_IC.filter(ee.Filter.eq('system:index', ee.String(y.format()))).first()
    fallback = GHSL_IC.filter(ee.Filter.eq('system:index', '2020')).first()
    return ee.Image(ee.Algorithms.If(img, img, fallback))

def event_image_by_id(event_id):
    return ee.Image(GFD.filter(ee.Filter.eq('id', event_id)).first())

def parse_lonlat(s):
    lon_str, lat_str = [q.strip() for q in s.split(',')]
    return float(lon_str), float(lat_str)

# --- Visualize only this row's cluster ---
def show_single_csv_row(event_id, centroid_xy_str, urban_threshold=0.20):
    img = event_image_by_id(event_id)
    if img.getInfo() is None:
        raise ValueError("Event not found. Check event_id from CSV")

    geom    = img.geometry()
    flooded = img.select('flooded').eq(1).unmask(0).clip(geom)
    perm    = img.select('jrc_perm_water').eq(1).unmask(0).clip(geom)
    fnp     = flooded.And(perm.Not()).rename('fnp').unmask(0).clip(geom)

    evt_year  = ee.Number.parse(ee.Date(img.get('system:time_start')).format('YYYY'))
    ghsl_year = nearest_ghsl_year(evt_year)
    ghsl      = ghsl_image_for_year(ghsl_year)

    built_frac = ghsl.select('built_surface').divide(10000.0).clip(geom)
    urban_mask = built_frac.gte(urban_threshold).rename('urban').unmask(0).clip(geom)
    urban_flood = fnp.And(urban_mask).rename('uf').unmask(0).clip(geom)

    proj_base = ghsl.select('built_surface').projection()
    labels = (urban_mask
              .reproject(proj_base.atScale(2000))
              .selfMask()
              .connectedComponents(ee.Kernel.plus(1), maxSize=1024)
              .select('labels').toInt64().rename('label'))

    lon, lat = parse_lonlat(centroid_xy_str)
    pt = ee.Geometry.Point([lon, lat])
    sampled = labels.sample(region=pt, scale=30, numPixels=1).first()
    if sampled.getInfo() is None or sampled.get('label').getInfo() is None:
        raise ValueError("Centroid not inside labeled urban area. Try smaller scale.")
    lab = ee.Number(ee.Feature(sampled).get('label'))

    cluster_mask = labels.eq(lab).selfMask()
    urban_cluster   = urban_mask.updateMask(cluster_mask)
    flooded_cluster = urban_flood.updateMask(cluster_mask)

    m = geemap.Map()
    m.addLayer(ee.Image().paint(geom, 0, 2), {'palette': ['black']}, 'Event footprint')
    m.addLayer(urban_cluster, {'min': 0, 'max': 1}, 'Urban cluster')
    m.addLayer(flooded_cluster, {'min': 0, 'max': 1, 'palette': ['white','red']}, 'Flooded in cluster')
    m.centerObject(pt, 10)
    display(m)

    print("event_id:", event_id)
    print("centroid:", centroid_xy_str)
    print("GHSL year:", ghsl_year.getInfo())
    print("urban threshold:", urban_threshold)

# --- Run ---
show_single_csv_row(event_id, centroid_xy, urban_thr)


EE ready
Please upload your CSV (urban_flood_clusters_*.csv)…


Saving urban_flood_clusters_1.csv to urban_flood_clusters_1.csv
Using: urban_flood_clusters_1.csv


Map(center=[39.002300000000005, 125.72410000000002], controls=(WidgetControl(options=['position', 'transparent…

event_id: 1627
centroid: 125.7241,39.0023
GHSL year: 2000
urban threshold: 0.2
